In [1]:
pip install netCDF4

     |████████████████████████████████| 4.7MB 21.6MB/s 
     |████████████████████████████████| 296kB 44.8MB/s 


In [2]:
import os
import glob
import time
import datetime

import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib

In [3]:
# only when using colab
from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/MyDrive/Uni/9/'
# PATH =''

Mounted at /content/drive


In [4]:
class Timer():
    """
    A small class to measure time during training.
    """
    def __init__(self):
        self._start_time = None

    def start(self):
        """
        Start a new timer
        """
        self._start_time = time.perf_counter()

    def stop(self):
        """
        Stop the timer, and report the elapsed time
        """
        if self._start_time is None:
            print(f"Timer is not running. Use .start() to start it")
            return 0
    
        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        
        return elapsed_time

In [5]:
def load_weights(Model, weight_file, dataset, input_shape):
    """
    Builds the model by using the call method on input and then loads the weights.
    """
    for sequence, t in dataset.take(1):
        model2(sequence, training=False, input_shape=input_shape)
        
    return Model.load_weights()

# Hyperparameter

In [20]:
# data param
day_len = 30  # actual day length is day_len*stack_days
stack_days = 3  # average over stack to allow more day_len
batch_size = 16
total_months = (2100-2015+1)*12

data_models = ['GFDL-ESM4','IPSL-CM6A-LR','MPI-ESM1-2-HR']  # models for temp, prec, LAI
dmodel = data_models[1]  # TODO: choose a model randomly
scenarios = ['ssp126', 'ssp370', 'ssp585']
scenario = scenarios[0]

# model param
num_filters = 16

# training param
epochs = 50
learning_rate = 0.001
training_name = "Version_x"

# Training data
Ideally, load all data for training and prediction at once, but doesn't fit into memory. Use separate loading scheme for training and deployment instead.

naming convention: dx=dataarray, ds=dataset, dn=numpy array. Notice that we save numpy array explictly to save loading time inside generator

Use separate data for test and validation set. We use the last 14 years for validation.

In [21]:
# compute average over every stack_days days
average_days = lambda dn : np.mean(
    np.reshape(dn[:(dn.shape[0]//stack_days)*stack_days],  # cut out left out few days
               (-1,stack_days,dn.shape[-2],dn.shape[-1])),
               axis=1)

# climate historical
dn_temp = average_days(np.array(xr.open_mfdataset(PATH+'data/near_surface_air_temperature/historical/{}/*.nc'.format(dmodel)).tas))
dn_prec = average_days(np.array(xr.open_mfdataset(PATH+'data/precipitation_flux/historical/{}/*.nc'.format(dmodel)).pr))

# vegetation historical
npp_files = glob.glob(PATH+'data/net_primary_production_on_land/historical/**/*.nc', recursive=True) # TODO: use all models
dn_npp = np.nan_to_num(np.array(xr.open_mfdataset(np.random.choice(np.array(npp_files))).npp)) 

dx_lai = xr.open_mfdataset(PATH+'data/leaf_area_index/historical/{}/*.nc'.format(dmodel)).lai  # not numpy as time index needed
dx_lai = dx_lai.fillna(0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  
/usr/local/lib/python3.6/dist-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdata

### Input normalization

##### Axis
---

- (x) Consider each location as an independent time series: We assume some regions to have overall higher vegetation than others, this information should be kept

- (x) Consider each time slice as an independent image: We assume some seasons to have higher vegetation than others

- (o) Normalize over all axis (time, lat, lon)

##### Range
---
- Z-standardize inputs, (0,1)
- minmax scale outpus (using tanh)

In [22]:
# define train and validation separation
train_max_index = dn_temp.shape[0] - (365*14)

maxi_in = 10  # used to decide scale of minmax scaling
mini_in = 0
maxi_out = 10
mini_out = 0

# Types of normalization
z_standard = lambda x : (x - np.mean(x)) / np.std(x)
minmax_scale = lambda x, mini_a, maxi_a, min_b, max_b : mini_a + (maxi_a-mini_a) * (x-min_b)/(max_b-min_b)

# Input normalization
temp_min = np.min(dn_temp[:train_max_index])  # normalize using means only from training data
temp_max = np.max(dn_temp[:train_max_index])
dn_temp = minmax_scale(dn_temp, mini_in, maxi_in, temp_min, temp_max)

prec_min = np.min(dn_prec[:train_max_index])
prec_max = np.max(dn_prec[:train_max_index])
dn_prec = minmax_scale(dn_prec, mini_in, maxi_in, prec_min, prec_max)

# Output normalization, use unshortend index
npp_min = np.min(dn_npp[:train_max_index*stack_days])
npp_max = np.max(dn_npp[:train_max_index*stack_days])
dn_npp = minmax_scale(dn_npp, mini_in, maxi_in, npp_min, npp_max)

lai_min = np.min(dx_lai[:train_max_index*stack_days])  
lai_max = np.max(dx_lai[:train_max_index*stack_days])
dx_lai = minmax_scale(dx_lai, mini_out, maxi_out, lai_min, lai_max)

### Generator


In [23]:
# range of output month indicies needed for plotting and generator
val_months = 12*14 # 14 years of validation data

train_max_month = 1978-val_months
train_min_month = day_len//28

val_max_month = 1978
val_min_month = train_max_month +1

In [24]:
def train_gen_data_card():
    while True:
        # array to append to
        endstamp = []
        output_day_i = np.zeros(batch_size)
        lai = np.zeros((batch_size, dn_npp.shape[1], dn_npp.shape[2]))  # batch, lon, lat
        npp = np.zeros((batch_size, dn_npp.shape[1], dn_npp.shape[2]))
        temp = np.zeros((batch_size, day_len, dn_npp.shape[1], dn_npp.shape[2]))  # batch, time, lon, lat
        prec = np.zeros((batch_size, day_len, dn_npp.shape[1], dn_npp.shape[2]))
        
        # index of output in month
        output_month_i = np.random.randint(train_min_month, train_max_month, size=batch_size)  # y_pred timepoint in int

        # convert output index to timestamp
        try:
            for i in range(batch_size):
                endstamp.append(dx_lai.indexes['time'].to_datetimeindex()[output_month_i[i]])  # cfttimeindex to datetime               
        except:
            for i in range(batch_size):
                endstamp.append(dx_lai.indexes['time'][output_month_i[i]])

        # convert output month index to day index
        for i in range(batch_size):
            output_day_i[i] = (endstamp[i] - pd.Timestamp('1850-01-01T12')).days  # output is i-th day in int
        output_day_i = np.int_(output_day_i) // stack_days  # 0,1,2 is stacked onto 0, 3,4,5 to 1 etc.

        # save month-based time slices
        lainp = np.array(dx_lai)
        for i in range(batch_size):
            lai[i] = lainp[output_month_i[i]]
        for i in range(batch_size):
            npp[i] = dn_npp[output_month_i[i]]

        # day-based metrics
        for i in range(batch_size):
            temp[i] = dn_temp[output_day_i[i]-day_len:output_day_i[i]]
        for i in range(batch_size):
            prec[i] = dn_prec[output_day_i[i]-day_len:output_day_i[i]]

        # merge features
        inputs = np.stack((temp,prec), axis=-1)  # two features
        outputs = np.stack((lai,npp), axis=-1)

        yield (inputs, outputs)

In [25]:
def val_gen_data_card():
    while True:
        # array to append to
        endstamp = []
        output_day_i = np.zeros(batch_size)
        
        lai = np.zeros((batch_size, dn_npp.shape[1], dn_npp.shape[2]))  # batch, lon, lat
        npp = np.zeros((batch_size, dn_npp.shape[1], dn_npp.shape[2]))
        temp = np.zeros((batch_size, day_len, dn_npp.shape[1], dn_npp.shape[2]))  # batch, time, lon, lat
        prec = np.zeros((batch_size, day_len, dn_npp.shape[1], dn_npp.shape[2]))
        
        # index of output in month
        output_month_i = np.random.randint(val_min_month, val_max_month, size=batch_size)  # y_pred timepoint in int

        # convert output index to timestamp
        try:
            for i in range(batch_size):
                endstamp.append(dx_lai.indexes['time'].to_datetimeindex()[output_month_i[i]])  # cfttimeindex to datetime               
        except:
            for i in range(batch_size):
                endstamp.append(dx_lai.indexes['time'][output_month_i[i]])

        # convert output month index to day index
        for i in range(batch_size):
            output_day_i[i] = (endstamp[i] - pd.Timestamp('1850-01-01T12')).days  # output is i-th day in int
        output_day_i = np.int_(output_day_i)

        # save month-based time slices
        lainp = np.array(dx_lai)
        for i in range(batch_size):
            lai[i] = lainp[output_month_i[i]]
        for i in range(batch_size):
            npp[i] = dn_npp[output_month_i[i]]

        # day-based metrics
        for i in range(batch_size):
            temp[i] = dn_temp[output_day_i[i]-day_len:output_day_i[i]]
        for i in range(batch_size):
            prec[i] = dn_prec[output_day_i[i]-day_len:output_day_i[i]]

        # merge features
        inputs = np.stack((temp,prec), axis=-1)  # two features
        outputs = np.stack((lai,npp), axis=-1)

        yield (inputs, outputs)

### tf.Dataset

In [26]:
# Create tf.Dataset objects
train_ds = tf.data.Dataset.from_generator(train_gen_data_card, output_types = (tf.float32,tf.float32))
train_ds = train_ds.take(30).prefetch(tf.data.experimental.AUTOTUNE)

val_ds = tf.data.Dataset.from_generator(val_gen_data_card, output_types = (tf.float32,tf.float32))
val_ds = val_ds.take(30).prefetch(tf.data.experimental.AUTOTUNE)

# Model

In [27]:
class ConvLSTM(tf.keras.Model):
    def __init__(self, num_filters):
        super(ConvLSTM, self).__init__()
        
        do = 0
        self.convlstm2D_1 = tf.keras.layers.ConvLSTM2D(filters = num_filters, kernel_size=(3,3),
                                                     padding="same", return_sequences=True,
                                                      activation = "tanh",dropout= do)
        self.bn_1 = tf.keras.layers.BatchNormalization()
        #self.acti_1 = tf.keras.layers.Activation(activation)
        
        self.convlstm2D_2 = tf.keras.layers.ConvLSTM2D(filters = num_filters, kernel_size=(3,3),
                                                     padding="same", return_sequences=True,
                                                      activation = "tanh",dropout=do)
        self.bn_2 = tf.keras.layers.BatchNormalization()
        #self.acti_2 = tf.keras.layers.Activation(activation)


        self.convlstm2D_3 = tf.keras.layers.ConvLSTM2D(filters = num_filters, kernel_size=(3,3),
                                                     padding="same", return_sequences=True,
                                                      activation = "tanh",dropout=do)
        self.bn_3 = tf.keras.layers.BatchNormalization()
        #self.acti_3 = tf.keras.layers.Activation(activation)
        
        self.convlstm2D_4 = tf.keras.layers.ConvLSTM2D(filters = num_filters, kernel_size=(3,3),
                                                     padding="same", return_sequences=True,
                                                      activation = "tanh",dropout=do)
        self.bn_4 = tf.keras.layers.BatchNormalization()
        #self.acti_4 = tf.keras.layers.Activation(activation)
        
        # convolve over time, lat, lon. This means that we assume timesteps close to each other share local similarities
        self.conv3d = tf.keras.layers.Conv3D(filters = 2, kernel_size = (3,3,3), 
                                             activation= "tanh", padding="same")
        
        self.do3d = tf.keras.layers.Dropout(do)
        # computed convolved sum over all time dimension to get a single time slice
        self.bottleneck = tf.keras.layers.Conv3D(filters=1, kernel_size=1, activation="relu",strides=1)


    def call(self, x, training, input_shape):
        # (batch, time, lat, lon, channel)
        x = tf.ensure_shape(x, input_shape) 
        # (batch, time, lat, lon, channel)
        x = self.convlstm2D_1(x,training= training)
        # (batch, time, lat1, lon1, filter1)
        x = self.bn_1(x,training = training)
        
        x = self.convlstm2D_2(x,training = training)
        x = self.bn_2(x,training = training)
        
        x = self.convlstm2D_3(x,training = training)
        x = self.bn_3(x,training = training)
        
        x = self.convlstm2D_4(x,training = training)
        x = self.bn_4(x, training = training)
        # (batch, time, lat4, lon4, filter4)
        x = self.do3d(x,training= training)
        x = self.conv3d(x)
        # (batch, newtime, newlat, newlon, newfilter=2)
        
        x = tf.transpose(x, [0,4,2,3,1])
        # (batch, 2, lat, lon, time)
        x = self.bottleneck(x)
        # (batch, 2, lat, lon, 1)
        
        x = tf.transpose(x, [0,4,2,3,1])
        # (batch, 1, lat, lon, 2)
        x = tf.squeeze(x,axis=1)
        # (batch, lat, lon, 2)
        
        return x

In [28]:
model = ConvLSTM(num_filters=num_filters)
input_shape = (batch_size, day_len, 36, 72, 2)
# model.build(input_shape)
# model.summary() # TODO: doesn't work cuz some layers aren't built?
# TODO: The fact that we can't build the model probably results in undefined rank error

# Training

In [29]:
@tf.function
def train_step(model, data, loss_function, optimizer, train_loss_metric, input_shape=input_shape):
    '''
    Training for one epoch.
    '''
    for img, target in train_ds:
        # forward pass with GradientTape
        with tf.GradientTape() as tape:
            prediction = model(img, training=True, input_shape=input_shape)
            loss = loss_function(target, prediction) + tf.reduce_sum(model.losses)
            # TODO: does model have reg loss if we don't use kernel reg?

        # backward pass via GradienTape (auto-gradient calc)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # update metrics
        train_loss_metric.update_state(loss)
        
#@tf.function        
def eval_step(model, ds, loss_function, loss_metric, input_shape=input_shape):
    '''
    Evaluation Loop.
    '''
    for sequence, target in ds:
        # forward pass
        prediction = model(sequence, training=False, input_shape=input_shape)
        # update metrics
        loss = loss_function(target, prediction)
        loss_metric.update_state(loss)
        
    fig, axe = plt.subplots(2,3, figsize=(25,10))
    plt.tight_layout(pad= 0.05)
    axe[0,0].imshow(target[0,:,:,0], cmap='gray', origin='lower')
    axe[0,1].imshow(prediction[0,:,:,0], cmap='gray', origin='lower')
    axe[1,0].imshow(target[0,:,:,1], cmap='gray', origin='lower')
    axe[1,1].imshow(prediction[0,:,:,1], cmap='gray', origin='lower')
    axe[0,2].imshow(np.abs(target[0,:,:,0]-prediction[0,:,:,0]), cmap = "bwr", 
                    origin= "lower", vmin = 0,vmax =maxi_out)
    axe[1,2].imshow(np.abs(target[0,:,:,1]-prediction[0,:,:,1]), cmap = "bwr", 
                    origin= "lower", vmin = 0,vmax =maxi_out)


# TODO: maybe we want another function (predict) to deploy the model to predict future vegetation and automatically save?

In [30]:
timer = Timer()

# define loss
loss_function = tf.keras.losses.MSE
# define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate)

# 2nd-order metric to take mean over all samples
train_loss_metric = tf.keras.metrics.Mean('train_loss')
val_loss_metric = tf.keras.metrics.Mean('val_loss')

# initialize the logger for Tensorboard visualization
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train_ConvLSTM'    
val_log_dir = 'logs/gradient_tape/' + current_time + '/val_ConvLSTM'       
train_summary_writer = tf.summary.create_file_writer(train_log_dir)  
val_summary_writer = tf.summary.create_file_writer(val_log_dir)

times = []

In [31]:
tf.keras.backend.clear_session()

for epoch in range(epochs):
    print(f'\n[EPOCH] ____________________{epoch}____________________')
    
    # training step with metrics update--------------------------------------------------------
    timer.start()
    train_step(model, train_ds, loss_function, optimizer, train_loss_metric)
    train_loss = train_loss_metric.result()
    with train_summary_writer.as_default():     # logging our metrics to a file which is used by tensorboard
        tf.summary.scalar('loss', train_loss, step=epoch)
    elapsed_time = timer.stop()
    print(f'[{epoch}] - Finished Epoch in {elapsed_time:0.2f} seconds - train_loss: {train_loss:0.4f}')
    
    # evaluation step with metrics update--------------------------------------------------------
    timer.start()
    eval_step(model, val_ds, loss_function, loss_metric=val_loss_metric)
    plt.show()
    val_loss = val_loss_metric.result()    
    with val_summary_writer.as_default():       # logging our metrics to a file which is used by tensorboard
        tf.summary.scalar('loss', val_loss, step=epoch)    
    print(f'\n[{epoch}] - Finished evaluation - val_loss: {val_loss:0.4f}')#, val_accuracy: {val_acc:0.4f}')
    
    # Resetting train and validation metrics-----------------------------------------------------
    train_loss_metric.reset_states()
    val_loss_metric.reset_states()
    elapsed_time = timer.stop()
    times.append(elapsed_time)
  
    if epoch%3 == 0:
        print(f'\n[INFO] - Total time elapsed: {np.sum(times)/60:0.4f} min. Total time remaining: {(np.sum(times)/(epoch+1))*(epochs-epoch-1)/60:0.4f} min.')

print(f'[INFO] - Total run time: {np.sum(times)/60:0.4f} min.')


[EPOCH] ____________________0____________________
[0] - Finished Epoch in 38.44 seconds - train_loss: 1.1600


InvalidArgumentError: ignored

In [ ]:
# save weights of model
model.save_weights("model_weights/"+ training_name + ".hdf5",save_format="hdf5",overwrite=False)

# Prediction data
one could also use just function. Systematically sliding window to predict all future months using timestamp index

In [ ]:
# climate historical
dx_temp = xr.open_mfdataset(PATH+'data/near_surface_air_temperature/historical/{}/*.nc'.format(dmodel)).tas
dx_prec = xr.open_mfdataset(PATH+'data/precipitation_flux/historical/{}/*.nc'.format(dmodel)).pr

# climate future
dx_temp_future = xr.open_mfdataset(PATH+'data/near_surface_air_temperature/{}/{}/*.nc'.format(scenario, dmodel)).tas
dx_temp_future.tas = minmax_scale(dx_temp_future.tas, mini_in, maxi_in, temp_min, temp_max)  # scale with training data minmax
dx_temp_future = xr.concat((dx_temp, dx_temp_future), dim='time')
dx_prec_future = xr.open_mfdataset(PATH+'data/precipitation_flux/{}/{}/*.nc'.format(scenario, dmodel)).pr
dx_temp_future.pr = minmax_scale(dx_temp_future.pr, mini_in, maxi_in, prec_min, prec_max)
dx_prec_future = xr.concat((dx_prec, dx_prec_future), dim='time')

In [ ]:
def gen_future_climate():
#     first_month = (2015-1850)*12 + 1  # int index of which month january 2015 is with 0 being january 1850

    counter = 0
    while counter < total_months: # predict from 2015 to 2100, excluding last month
        cyear = (counter//12) + 2015
        
        cmonth = (counter+1) % 12
        
        if cmonth == 0:
            cmonth = 12
        
        current_timestamp = pd.Timestamp(cyear, cmonth, 1)
        input_start_timestamp = current_timestamp - pd.Timedelta(day_len-1, unit='day')

        counter += 1
       
        data = np.stack((np.array(dx_temp_future.loc[input_start_timestamp:current_timestamp+pd.Timedelta(1,unit='day')]),
                         np.array(dx_prec_future.loc[input_start_timestamp:current_timestamp+pd.Timedelta(1,unit='day')])),
                         axis=-1)
        
        yield data
    

In [ ]:
# data pipeline
ds_future = tf.data.Dataset.from_generator(generator=gen_future_climate, 
                                           output_types=(tf.float32)).batch(batch_size)

In [ ]:
for i in ds_future.take(1):
    print(i.shape)

# Predict future

In [ ]:
# Load weights if using separately from training
# model = load_weights(PFAD)

In [ ]:
for data in ds_future:  # just batch whole data into one batch
    try:
        predictions = np.concatenate((predictions, model(data, training=False, input_shape=input_shape)), axis=0)    
    except Exception as e:
        predictions = model(data, training=False, input_shape=input_shape)

In [ ]:
np.save('results/pred_{}_{}'.format(training_name,scenario), predictions)

# Plot

In [ ]:
plt.rcParams["animation.html"] = "jshtml"  # allow animation for jupyter
plt.rcParams['xtick.bottom'] = False
plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.labelleft'] = False

In [ ]:
print(predictions.shape)

In [ ]:
frames = []  # append each image
fig = plt.figure()
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)

for timeindex in range(total_months):  # animate for 1 yr

    frames.append([plt.imshow(predictions[timeindex,:,:,0], # TODO: change to 1 to save NPP
                              cmap='gray', origin='lower', animated=True)])

ani = matplotlib.animation.ArtistAnimation(fig, frames, interval=100, blit=True, repeat=True)
# ani.save('figs/pred_v1_{}_LAI.gif'.format(scenario), writer='imagemagick', fps=60)
ani
